<a href="https://colab.research.google.com/github/chengruchou/RT-DETR-test/blob/main/RT_DETR_v2_finetune_on_a_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision
!git clone https://github.com/lyuwenyu/RT-DETR.git
%cd RT-DETR/rtdetrv2_pytorch
!pip install -r requirements.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install roboflow
%cd /content
from roboflow import Roboflow
rf = Roboflow(api_key="ynvPp1noiph3TmelaKST")
project = rf.workspace("scale-detection-mdjzo").project("wisard_vis")
version = project.version(1)
dataset = version.download("coco")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
/content
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to WiSARD_VIS-1 in coco:: 100%|██████████| 16064/16064 [00:04<00:00, 3874.14it/s]


In [4]:
import os
import shutil

# 資料集的根目錄
dataset_dir = '/content/WiSARD_VIS-1'

# 標註檔案的目標資料夾
annotations_dir = os.path.join(dataset_dir, '_annotations')

# 確保目標資料夾存在
os.makedirs(annotations_dir, exist_ok=True)

def move_and_rename_annotations(source_folder, prefix, target_folder, annotation_extensions=('.json', '.txt')):
    for filename in os.listdir(source_folder):
        if filename.endswith(annotation_extensions):
            source_file = os.path.join(source_folder, filename)
            new_filename = f"{prefix}_{filename}"
            target_file = os.path.join(target_folder, new_filename)
            shutil.move(source_file, target_file)
            print(f'已移動並重新命名 {source_file} 到 {target_file}')

subsets = ['train', 'test', 'valid']
for subset in subsets:
    subset_folder = os.path.join(dataset_dir, subset)
    move_and_rename_annotations(subset_folder, subset, annotations_dir)


已移動並重新命名 /content/WiSARD_VIS-1/train/_annotations.coco.json 到 /content/WiSARD_VIS-1/_annotations/train__annotations.coco.json
已移動並重新命名 /content/WiSARD_VIS-1/test/_annotations.coco.json 到 /content/WiSARD_VIS-1/_annotations/test__annotations.coco.json
已移動並重新命名 /content/WiSARD_VIS-1/valid/_annotations.coco.json 到 /content/WiSARD_VIS-1/_annotations/valid__annotations.coco.json


In [7]:
import torch,torchvision
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
!CUDA_VISIBLE_DEVICES=0 python /content/RT-DETR/rtdetrv2_pytorch/tools/train.py -c /content/RT-DETR/rtdetrv2_pytorch/configs/rtdetrv2/rtdetrv2_r50vd_6x_coco.yml --use-amp --seed=0

2025-03-30 17:00:32.509190: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743354032.759313    2989 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743354032.823620    2989 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-30 17:00:33.324077: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Not init distributed mode.
cfg:  {'task': 'detection', '_model': None, '_postprocessor': None, '_criterion': None, '_